In [23]:
import pandas as pd 
import numpy as np
import random 
from sklearn.preprocessing import MinMaxScaler
from deap import base, creator, tools
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
# Cargar datos desde un archivo JSON
with open('SeisBenchV1_v1_1.json', 'r') as json_file:
    data = pd.read_json(json_file)

In [4]:
data

,Type,f1_t_mean,f2_t_std,f3_t_var,f4_t_entropy,f5_t_kurtosis,f6_t_multiscaleEntropy,f7_t_time2peak,f8_t_rms,f9_t_peak2peak,...,f75_w_t_peak2peak_D5,f76_w_t_peak2peak_D6,f77_w_t_peak2rms_A6,f78_w_t_peak2rms_D1,f79_w_t_peak2rms_D2,f80_w_t_peak2rms_D3,f81_w_t_peak2rms_D4,f82_w_t_peak2rms_D5,f83_w_t_peak2rms_D6,f84_w_t_meanEnergyCoeff
0,VT,-0.000059,0.159283,0.025371,125.575049,8.304542,1.659625,1.49,0.159248,1.735696,...,0.194575,0.028046,10.062697,8.860736,7.890076,6.719720,4.237517,3.802709,8.048929,0.024310
1,LP,-0.000042,0.197845,0.039143,393.853185,5.290367,2.004580,10.06,0.197824,1.837824,...,0.587130,0.051686,19.361546,5.857056,4.538726,4.670957,5.671383,4.123603,3.331997,0.038853
2,LP,0.000037,0.190926,0.036453,396.682354,6.666304,1.744531,14.87,0.190909,1.934064,...,1.221593,0.058025,10.923910,4.332089,4.554143,6.152058,4.511500,7.012052,5.524468,0.035855
3,VT,-0.000049,0.363053,0.131808,620.945134,3.580948,1.614374,11.93,0.363016,1.807268,...,0.937122,0.139498,4.845511,11.707501,7.343261,5.640008,4.287812,5.132921,4.930505,0.128911
4,VT,-0.000039,0.217804,0.047439,235.107723,6.647344,1.567582,3.76,0.217769,1.730968,...,0.347865,0.028109,10.195495,9.866074,8.192508,5.847093,4.932813,4.201841,5.791239,0.045848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,LP,0.000012,0.178485,0.031857,159.060029,6.621999,1.886219,3.82,0.178446,1.873766,...,0.417810,0.093646,9.744392,4.498606,5.900042,5.808795,4.728805,4.308743,4.339143,0.031003
1183,LP,-0.000002,0.138430,0.019163,119.363546,11.461582,1.436148,7.51,0.138405,1.816508,...,0.172849,0.008826,12.986258,6.735413,7.479872,8.237423,5.291185,5.633831,3.608272,0.018447
1184,LP,-0.000003,0.172697,0.029824,282.899977,7.743663,1.660353,11.55,0.172678,1.895220,...,0.844632,0.396342,5.553231,3.709472,6.540323,5.570468,4.763939,4.938626,4.957961,0.029158
1185,LP,-0.000055,0.183042,0.033504,194.828120,9.098961,1.697572,3.85,0.183014,1.935800,...,0.640989,0.227931,9.421489,8.069422,6.638473,8.576695,5.664434,4.372382,6.493511,0.032539


In [32]:
data = data[~data['Type'].isin(['REGIONAL', 'HB', 'ICEQUAKE'])]
data

,Type,f1_t_mean,f2_t_std,f3_t_var,f4_t_entropy,f5_t_kurtosis,f6_t_multiscaleEntropy,f7_t_time2peak,f8_t_rms,f9_t_peak2peak,...,f75_w_t_peak2peak_D5,f76_w_t_peak2peak_D6,f77_w_t_peak2rms_A6,f78_w_t_peak2rms_D1,f79_w_t_peak2rms_D2,f80_w_t_peak2rms_D3,f81_w_t_peak2rms_D4,f82_w_t_peak2rms_D5,f83_w_t_peak2rms_D6,f84_w_t_meanEnergyCoeff
0,VT,-0.000059,0.159283,0.025371,125.575049,8.304542,1.659625,1.49,0.159248,1.735696,...,0.194575,0.028046,10.062697,8.860736,7.890076,6.719720,4.237517,3.802709,8.048929,0.024310
1,LP,-0.000042,0.197845,0.039143,393.853185,5.290367,2.004580,10.06,0.197824,1.837824,...,0.587130,0.051686,19.361546,5.857056,4.538726,4.670957,5.671383,4.123603,3.331997,0.038853
2,LP,0.000037,0.190926,0.036453,396.682354,6.666304,1.744531,14.87,0.190909,1.934064,...,1.221593,0.058025,10.923910,4.332089,4.554143,6.152058,4.511500,7.012052,5.524468,0.035855
3,VT,-0.000049,0.363053,0.131808,620.945134,3.580948,1.614374,11.93,0.363016,1.807268,...,0.937122,0.139498,4.845511,11.707501,7.343261,5.640008,4.287812,5.132921,4.930505,0.128911
4,VT,-0.000039,0.217804,0.047439,235.107723,6.647344,1.567582,3.76,0.217769,1.730968,...,0.347865,0.028109,10.195495,9.866074,8.192508,5.847093,4.932813,4.201841,5.791239,0.045848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,LP,0.000012,0.178485,0.031857,159.060029,6.621999,1.886219,3.82,0.178446,1.873766,...,0.417810,0.093646,9.744392,4.498606,5.900042,5.808795,4.728805,4.308743,4.339143,0.031003
1183,LP,-0.000002,0.138430,0.019163,119.363546,11.461582,1.436148,7.51,0.138405,1.816508,...,0.172849,0.008826,12.986258,6.735413,7.479872,8.237423,5.291185,5.633831,3.608272,0.018447
1184,LP,-0.000003,0.172697,0.029824,282.899977,7.743663,1.660353,11.55,0.172678,1.895220,...,0.844632,0.396342,5.553231,3.709472,6.540323,5.570468,4.763939,4.938626,4.957961,0.029158
1185,LP,-0.000055,0.183042,0.033504,194.828120,9.098961,1.697572,3.85,0.183014,1.935800,...,0.640989,0.227931,9.421489,8.069422,6.638473,8.576695,5.664434,4.372382,6.493511,0.032539


In [36]:
# Split data into X and y
y_data = data.iloc[:, 0]  # Select the first column as 'y'
X_data = data.iloc[:, 1:]

In [35]:
unique_outcomes = y_data.unique()
unique_outcomes

array(['VT', 'LP'], dtype=object)

#### Data normalization

In [7]:
# Normalize the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_data)

# Convert the normalized features into a dataframe
X_scaled = pd.DataFrame(X_scaled, columns=X_data.columns)
X_scaled.head()

,f1_t_mean,f2_t_std,f3_t_var,f4_t_entropy,f5_t_kurtosis,f6_t_multiscaleEntropy,f7_t_time2peak,f8_t_rms,f9_t_peak2peak,f10_t_peak2rms,...,f75_w_t_peak2peak_D5,f76_w_t_peak2peak_D6,f77_w_t_peak2rms_A6,f78_w_t_peak2rms_D1,f79_w_t_peak2rms_D2,f80_w_t_peak2rms_D3,f81_w_t_peak2rms_D4,f82_w_t_peak2rms_D5,f83_w_t_peak2rms_D6,f84_w_t_meanEnergyCoeff
0,0.308173,0.247948,0.125720,0.031442,0.227874,0.670681,0.004878,0.247849,0.420602,0.387087,...,0.071505,0.012634,0.343762,0.418231,0.391023,0.328678,0.161757,0.149867,0.518262,0.119583
1,0.322134,0.371598,0.218533,0.170931,0.095844,0.905458,0.050815,0.371548,0.644631,0.260358,...,0.254942,0.025112,0.767478,0.225288,0.116317,0.128347,0.358863,0.185590,0.091188,0.218376
2,0.386335,0.349410,0.200403,0.172402,0.156114,0.728468,0.076597,0.349372,0.855745,0.279310,...,0.551422,0.028458,0.383004,0.127331,0.117581,0.273171,0.199420,0.507141,0.289695,0.198006
3,0.316393,0.901336,0.843043,0.289005,0.020966,0.639883,0.060838,0.901256,0.577603,0.022293,...,0.418491,0.071464,0.106033,0.601095,0.346201,0.223102,0.168671,0.297951,0.235918,0.830167
4,0.324330,0.435596,0.274443,0.088392,0.155283,0.608036,0.017045,0.435503,0.410230,0.212444,...,0.143136,0.012667,0.349813,0.482810,0.415813,0.243351,0.257336,0.194300,0.313849,0.265895


#### Genetic algorithm

In [24]:
# Fitness function using ReliefF for feature selection
# Modify the evaluate function to accept a tuple (individual, X_data, X_scaled, y_data)
def evaluate_svm(args):
    individual, X_data, X_scaled, y_data = args
    selected_features = [
        feature for feature, include in zip(X_data.columns, individual) if include
    ]

    x_train, x_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

    # Create and train an SVM classifier
    svm = SVC(probability=True)
    svm.fit(x_train, y_train)

    # Predict probabilities for the test set
    y_pred_prob = svm.predict_proba(x_test)

    # Calculate the AUC score
    auc_score = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

    return (auc_score,)

In [18]:
def evaluate_ann(args):
    individual, X_data, X_scaled, y_data = args
    selected_features = [
        feature for feature, include in zip(X_data.columns, individual) if include
    ]

    x_train, x_test, y_train, y_test = train_test_split(
        X_data, y_data, test_size=0.2, random_state=42
    )

    # Create and train an SVM classifier
    mlp = MLPClassifier(
        hidden_layer_sizes=(84, 84),
        activation='relu',
        max_iter=100,
        random_state=42,
        learning_rate_init=0.001,
        #alpha=0.1,
        #beta_1=0.5,
        #beta_2=0.5
    )
    mlp.out_activation_ = 'softmax'

    mlp.fit(x_train, y_train)

    # Predict probabilities for the test set
    y_pred_prob = mlp.predict_proba(x_test)

    # Calculate the AUC score
    auc_score = roc_auc_score(y_test, y_pred_prob, multi_class="ovr")

    return (auc_score,)

In [40]:
def evaluate_nv(args):
    individual, X_data, X_scaled, y_data = args
    selected_features = [
        feature for feature, include in zip(X_data.columns, individual) if include
    ]

    x_train, x_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

    # Create and train an SVM classifier
    nv = GaussianNB()
    nv.fit(x_train, y_train)

    # Predict probabilities for the test set
    y_pred_prob = nv.predict_proba(x_test)

    # Calculate the AUC score
    auc_score = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

    return (auc_score,)

In [ ]:
def evaluate_knn(args):
    individual, X_data, X_scaled, y_data = args
    selected_features = [
        feature for feature, include in zip(X_data.columns, individual) if include
    ]

    x_train, x_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

    # Create and train an SVM classifier
    knn = 
    nv.fit(x_train, y_train)

    # Predict probabilities for the test set
    y_pred_prob = nv.predict_proba(x_test)

    # Calculate the AUC score
    auc_score = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')

    return (auc_score,)

In [20]:
from sklearn.neighbors import KNeighborsClassifier


def evaluate_knn(args):
    individual, X_data, X_scaled, y_data = args
    selected_features = [
        feature for feature, include in zip(X_data.columns, individual) if include
    ]

    x_train, x_test, y_train, y_test = train_test_split(
        X_data, y_data, test_size=0.2, random_state=42
    )

    # Create and train an SVM classifier
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(x_train, y_train)

    # Predict probabilities for the test set
    y_pred_prob = knn.predict_proba(x_test)

    # Calculate the AUC score
    auc_score = roc_auc_score(y_test, y_pred_prob, multi_class="ovr")

    return (auc_score,)

In [25]:
def evaluate_ada(args):
    individual, X_data, X_scaled, y_data = args
    selected_features = [
        feature for feature, include in zip(X_data.columns, individual) if include
    ]

    x_train, x_test, y_train, y_test = train_test_split(
        X_data, y_data, test_size=0.2, random_state=42
    )

    base_classifier = DecisionTreeClassifier(max_depth=1)

    # Create AdaBoost classifier
    adaboost_classifier = AdaBoostClassifier(base_classifier, n_estimators=50, random_state=42)

    # Train the AdaBoost classifier
    adaboost_classifier.fit(x_train, y_train)

    # Make predictions on the test set
    y_probs = adaboost_classifier.predict_proba(x_test)

    # Evaluate the AUC score
    auc_score = roc_auc_score(y_test, y_probs)

    return (auc_score,)

In [30]:
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression


def evaluate_ensemble(args):
    individual, X_data, X_scaled, y_data = args
    selected_features = [
        feature for feature, include in zip(X_data.columns, individual) if include
    ]

    x_train, x_test, y_train, y_test = train_test_split(
        X_data, y_data, test_size=0.2, random_state=42
    )

    # Define base classifiers
    clf1 = RandomForestClassifier(random_state=42)
    clf2 = GradientBoostingClassifier(random_state=42)
    clf3 = SVC(probability=True, random_state=42)
    clf4 = LogisticRegression(random_state=42)

    # Create an ensemble classifier
    ensemble_clf = VotingClassifier(estimators=[('rf', clf1), ('gb', clf2), ('svc', clf3), ('lr', clf4)], voting='soft')

    # Fit the ensemble classifier on the training data
    ensemble_clf.fit(x_train, y_train)

    # Make probability predictions on the test data
    y_probs = ensemble_clf.predict_proba(x_test)


    # Evaluate the performance using AUC
    auc_score = roc_auc_score(y_test, y_probs, multi_class='ovr')

    return (auc_score,)

In [31]:
individual = []
for i in range(len(X_scaled.columns)):
    individual.append(1)
    
print(len(individual))

print(evaluate_ensemble((individual, X_data, X_scaled, y_data)))

84
(0.6197928401839616,)


In [9]:
# Create a fitness class
creator.create('FitnessMax', base.Fitness, weights=(1.0,))

# Create an individual class
creator.create('Individual', list, fitness=creator.FitnessMax)

# Create a toolbox
toolbox = base.Toolbox()

# Create a random bit generator
toolbox.register('attr_bool', random.randint, 0, 1)

# Initialize a population
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(X_scaled.columns))

# Define the population to be a list of individuals
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

# Register the evaluation operator
toolbox.register('evaluate', evaluate_nv)

# Register the crossover operator
toolbox.register('mate', tools.cxTwoPoint)

# Register a mutation operator
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)

# Register the selection operator
toolbox.register('select', tools.selTournament, tournsize=3)

# Create an initial population of 500 individuals
population = toolbox.population(n=50)

# Define probabilities of crossing and mutating
probab_crossing, probab_mutating = 0.5, 0.2

In [10]:
# Number of cores to use for parallel processing
from multiprocessing import Pool
import concurrent.futures

num_processes = 8  # Adjust this based on your system's capabilities
pool = Pool(processes=num_processes)

In [11]:
num_generations = 75

# Evaluate the entire population in parallel
fitnesses = list(pool.map(toolbox.evaluate, [(ind, X_data, X_scaled, y_data) for ind in population]))

# Set the fitness values of the population
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

# Close the pool of workers
pool.close()
pool.join()

# Iterate through generations
for g in range(num_generations):
    print(f'-- Generation {g} --')
    
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))
    
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        
        # Cross two individuals
        if random.random() < probab_crossing:
            toolbox.mate(child1, child2)
            
            # Delete the fitness values of the children
            del child1.fitness.values
            del child2.fitness.values
            
    # Apply mutation
    for mutant in offspring:
        if random.random() < probab_mutating:
            toolbox.mutate(mutant)
            del mutant.fitness.values
            
    # Evaluate the individuals with an invalid fitness in parallel
    invalid_individuals = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = list(pool.map(toolbox.evaluate, [(ind, X_data, X_scaled, y_data) for ind in invalid_individuals]))

    # Set the fitness values of the evaluated individuals
    for ind, fit in zip(invalid_individuals, fitnesses):
        ind.fitness.values = fit
        
    print(f'Evaluated {len(invalid_individuals)} individuals')
    
    # The population is entirely replaced by the offspring
    population[:] = offspring
    
    # Gather all the fitnesses in one list and print the stats
    fitnesses = [ind.fitness.values[0] for ind in population]
    
    length = len(population)
    mean = sum(fitnesses) / length
    sum2 = sum(x*x for x in fitnesses)
    std = abs(sum2 / length - mean**2)**0.5
    
    print(f'Min = {min(fitnesses)}, Max = {max(fitnesses)}')
    print(f'Mean = {mean}, Std = {std}')
    
    best_individual = tools.selBest(population, k=1)[0]
    print(f'Best individual is {best_individual}, {best_individual.fitness.values}')

In [ ]:
best_individual = tools.selBest(population, 1)[0]
selected_features = [feature for feature, include in zip(X_scaled.columns, best_individual) if include]
X_selected = X_scaled[selected_features]
X_selected.head()

,f1_t_mean,f3_t_var,f5_t_kurtosis,f6_t_multiscaleEntropy,f9_t_peak2peak,f10_t_peak2rms,f13_t_PeaksAboveRMSDensity_fun,f14_f_peaks_pos_1,f15_f_90_percent_energy,f16_f_entropy,...,f72_w_t_peak2peak_D2,f73_w_t_peak2peak_D3,f74_w_t_peak2peak_D4,f75_w_t_peak2peak_D5,f76_w_t_peak2peak_D6,f78_w_t_peak2rms_D1,f80_w_t_peak2rms_D3,f82_w_t_peak2rms_D5,f83_w_t_peak2rms_D6,f84_w_t_meanEnergyCoeff
0,0.308173,0.125720,0.227874,0.670681,0.420602,0.387087,0.335508,0.378641,0.463529,0.947019,...,0.629229,0.488480,0.240066,0.071505,0.012634,0.418231,0.328678,0.149867,0.518262,0.119583
1,0.322134,0.218533,0.095844,0.905458,0.644631,0.260358,0.408386,0.116505,0.207059,0.955788,...,0.185582,0.266061,0.712985,0.254942,0.025112,0.225288,0.128347,0.185590,0.091188,0.218376
2,0.386335,0.200403,0.156114,0.728468,0.855745,0.279310,0.222889,0.121359,0.054118,0.914255,...,0.060309,0.211387,0.579142,0.551422,0.028458,0.127331,0.273171,0.507141,0.289695,0.198006
3,0.316393,0.843043,0.020966,0.639883,0.577603,0.022293,0.524128,0.208738,0.308235,0.985715,...,0.808419,0.996958,0.831907,0.418491,0.071464,0.601095,0.223102,0.297951,0.235918,0.830167
4,0.324330,0.274443,0.155283,0.608036,0.410230,0.212444,0.271957,0.233010,0.320000,0.938802,...,0.630099,0.759558,0.458917,0.143136,0.012667,0.482810,0.243351,0.194300,0.313849,0.265895


In [ ]:
def customRand():
    # Check if there's a 'last_position.txt' file to resume from
    try:
        with open('last_position.txt', 'r') as last_position_file:
            last_position = int(last_position_file.read())
    except FileNotFoundError:
        last_position = 0  # Start from the beginning if no last position is saved

    # Read numbers from the text file starting from the last position
    with open('new_random_numbers.txt', 'r') as file:
        numbers = [int(line) for line in file.readlines()]

    # Normalize the numbers to the range [0, 1]
    normalized_numbers = [number / 4294967295 for number in numbers]

    # Determine the number of elements remaining to be read
    num_elements = len(normalized_numbers)
    remaining_elements = num_elements - last_position

    if remaining_elements == 0:
        raise ValueError("No remaining elements in the file.")

    # Get the corresponding normalized number
    random_number = normalized_numbers[last_position]

    # Update the last position
    last_position = last_position + 1

    # Save the new last position to 'last_position.txt'
    with open('last_position.txt', 'w') as last_position_file:
        last_position_file.write(str(last_position))

    return random_number

In [15]:
from sklearn.calibration import label_binarize
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt

# Instantiate a random forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform stratified k-fold cross-validation
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
auc_list = []
f1_list = []

all_y_probs = []
all_y_true = []

for train_index, test_index in stratified_kfold.split(X_scaled, y_data):
    X_train, X_test = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
    y_train, y_test = y_data.iloc[train_index], y_data.iloc[test_index]

    # Train the classifier
    rf_clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = rf_clf.predict(X_test)
    y_probs = rf_clf.predict_proba(X_test)

    all_y_probs.extend(y_probs)  # Assuming binary classification
    all_y_true.extend(y_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    auc = roc_auc_score(y_test, y_probs, multi_class='ovr')
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)

    # Append metrics to the lists
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    auc_list.append(auc)
    f1_list.append(f1)

# Calculate the mean and standard deviation of the metrics across folds
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)

mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)

mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)

mean_auc = np.mean(auc_list)
std_auc = np.std(auc_list)

mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Print the mean and standard deviation of the metrics
print(f'Mean Accuracy: {mean_accuracy:.2f} (Std: {std_accuracy:.2f})')
print(f'Mean Precision: {mean_precision:.2f} (Std: {std_precision:.2f})')
print(f'Mean Recall: {mean_recall:.2f} (Std: {std_recall:.2f})')
print(f'Mean AUC: {mean_auc:.2f} (Std: {std_auc:.2f})')
print(f'Mean F1 score: {mean_f1:.2f} (Std: {std_f1:.2f})')


# Convert the lists to NumPy arrays
all_y_probs = np.array(all_y_probs)
all_y_true = np.array(all_y_true)

# Binarize the true labels for ROC curve
all_y_true_binarized = label_binarize(all_y_true, classes=np.unique(all_y_true))

# Save AUC-ROC curve plot for each class
n_classes = len(np.unique(y_data))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(all_y_true_binarized[:, i], all_y_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve - Class {}'.format(i))
    plt.legend(loc="lower right")
    plt.savefig('roc_curve_class_{}.png'.format(i))


# Save Precision-Recall curve plot
precision, recall, _ = precision_recall_curve(all_y_true, all_y_probs)
plt.figure()
plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.savefig('precision_recall_curve.png')



Mean Accuracy: 0.95 (Std: 0.02)
Mean Precision: 0.95 (Std: 0.02)
Mean Recall: 0.95 (Std: 0.02)
Mean AUC: 0.79 (Std: 0.04)
Mean F1 score: 0.94 (Std: 0.02)


TypeError: 'numpy.float64' object is not callable